# Markov Model in Enrollment Prediction
### Sheng Liu, Haichao Wu
### Monday, Nov.  30, 2016

In [2]:
import pandas as pd
import numpy as np
import warnings
from sklearn import preprocessing
warnings.filterwarnings('ignore')
data = pd.read_csv('../CSD20_Resident_Data_Phase_1.csv', header = 0, sep = ',')#, index_col = 0)
data['Year'] = np.round(data['School Year']/10000)

<img src="img2.jpeg",width=800,height=800>

<img src="img1.jpeg",width=800,height=800>

<img src="img3.jpeg",width=800,height=800>

In [213]:
census_tract = 20
census20_data = data[data['2010 Census Tract']== census_tract]
A_dict = {}
B_dict = {}
for year in np.unique(census20_data['Year']):
    print(year)
    A = np.zeros([5,6])
    B = np.zeros(5)
    Census20_data = census20_data[census20_data['Year'] == year]
    Census20_data_next_year = census20_data[census20_data['Year'] == year+1]
    A[0][0] = int(Census20_data[Census20_data['Grade Level']=='K']['Count of Students'].values)
    A[0][1] = int(Census20_data[Census20_data['Grade Level']=='1']['Count of Students'].values)
    A[1][1] = int(Census20_data[Census20_data['Grade Level']=='1']['Count of Students'].values)
    A[1][2] = int(Census20_data[Census20_data['Grade Level']=='2']['Count of Students'].values)
    A[2][2] = int(Census20_data[Census20_data['Grade Level']=='2']['Count of Students'].values)
    A[2][3] = int(Census20_data[Census20_data['Grade Level']=='3']['Count of Students'].values)
    A[3][3] = int(Census20_data[Census20_data['Grade Level']=='3']['Count of Students'].values)
    A[3][4] = int(Census20_data[Census20_data['Grade Level']=='4']['Count of Students'].values)
    A[4][4] = int(Census20_data[Census20_data['Grade Level']=='4']['Count of Students'].values)
    A[:,5] = 80
    #A[0][6] = -int(Census20_data[Census20_data['Grade Level']=='1']['Count of Students'].values)
    #A[1][6] = -int(Census20_data[Census20_data['Grade Level']=='2']['Count of Students'].values)
    #A[2][6] = -int(Census20_data[Census20_data['Grade Level']=='3']['Count of Students'].values)
    #A[3][6] = -int(Census20_data[Census20_data['Grade Level']=='4']['Count of Students'].values)
    #A[4][6] = -int(Census20_data[Census20_data['Grade Level']=='5']['Count of Students'].values)
    if np.size(Census20_data_next_year) > 0:
        B = Census20_data_next_year['Count of Students'].values[1:] + Census20_data['Count of Students'].values[1:]
        B_dict[int(year-2000)] = B
    A_dict[int(year-2000)] = A
    
    
    

2001.0
2002.0
2003.0
2004.0
2005.0
2006.0
2007.0
2008.0
2009.0
2010.0


In [189]:
A[0][0] = int(Census20_data[Census20_data['Grade Level']=='K']['Count of Students'].values)
A[0][1] = int(Census20_data[Census20_data['Grade Level']=='1']['Count of Students'].values)
A[1][1] = int(Census20_data[Census20_data['Grade Level']=='1']['Count of Students'].values)
A[1][2] = int(Census20_data[Census20_data['Grade Level']=='2']['Count of Students'].values)
A[2][2] = int(Census20_data[Census20_data['Grade Level']=='2']['Count of Students'].values)
A[2][3] = int(Census20_data[Census20_data['Grade Level']=='3']['Count of Students'].values)
A[3][3] = int(Census20_data[Census20_data['Grade Level']=='3']['Count of Students'].values)
A[3][4] = int(Census20_data[Census20_data['Grade Level']=='4']['Count of Students'].values)
A[4][4] = int(Census20_data[Census20_data['Grade Level']=='4']['Count of Students'].values)
A[:,5] = 50
#A[0][6] = -int(Census20_data[Census20_data['Grade Level']=='1']['Count of Students'].values)
#A[1][6] = -int(Census20_data[Census20_data['Grade Level']=='2']['Count of Students'].values)
#A[2][6] = -int(Census20_data[Census20_data['Grade Level']=='3']['Count of Students'].values)
#A[3][6] = -int(Census20_data[Census20_data['Grade Level']=='4']['Count of Students'].values)
#A[4][6] = -int(Census20_data[Census20_data['Grade Level']=='5']['Count of Students'].values)

In [216]:
A_sum = np.zeros([6,6])
bA_sum =np.zeros(6)
for key in range(1,6):
    A_sum += np.dot(A_dict[key].T, A_dict[key])

for key in range(1,6):
    bA_sum = bA_sum + np.dot(B_dict[key],A_dict[key])    

In [217]:
from cvxopt import matrix, solvers
m = matrix(A_sum)
n = matrix(bA_sum)

In [206]:
m

<6x6 matrix, tc='d'>

In [207]:
n

<6x1 matrix, tc='d'>

In [218]:
sol=solvers.qp(m, -n)

In [219]:
print(sol['x'])

[ 8.29e-01]
[ 5.99e-01]
[ 8.59e-01]
[ 3.71e-01]
[ 9.27e-01]
[ 4.99e-02]



In [210]:
l = matrix(A_dict[5])
k = matrix(B_dict[5])
print(l*sol['x']- k)

[ 3.92e-01]
[ 1.75e-01]
[ 4.37e-01]
[ 1.48e+00]
[-5.37e+00]



<7x1 matrix, tc='d'>

array([[   148.,    146.,      0.,      0.,      0.,   1300.,   -146.],
       [   146.,    388.,    149.,      0.,      0.,   3000.,   -343.],
       [     0.,    149.,    334.,    122.,      0.,   2500.,   -289.],
       [     0.,      0.,    122.,    286.,    130.,   2500.,   -273.],
       [     0.,      0.,      0.,    130.,    312.,   2600.,   -290.],
       [  1300.,   3000.,   2500.,   2500.,   2600.,  62500.,  -6550.],
       [  -146.,   -343.,   -289.,   -273.,   -290.,  -6550.,    817.]])